In [ ]:
!pip install transformers accelerate evaluate datasets peft -q

In [ ]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

In [ ]:
model_name = "/home/rion/phi_3_lora/Phi-3-mini-4k-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
model.train() 


model.gradient_checkpointing_enable()


model = prepare_model_for_kbit_training(model)
     


config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=["qkv_proj", "o_proj", "up_gate_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    rank_pattern={"qkv_proj": 192, "up_gate_proj": 128}
)


model = get_peft_model(model, config)


model.print_trainable_parameters()

In [ ]:
data = load_dataset("parquet", data_files={'train': './dataset/phi_3_instruct_format.parquet'})

In [ ]:
def tokenize_function(examples):
    text = examples["examples"]

    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

tokenized_data = data.map(tokenize_function, batched=True)
     

tokenizer.pad_token = tokenizer.eos_token
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
lr = 2e-4
batch_size = 4
num_epochs = 10


training_args = transformers.TrainingArguments(
    output_dir= "check",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",

)

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    args=training_args,
    data_collator=data_collator
)

model.config.use_cache = False 
trainer.train()


model.config.use_cache = True